In [ ]:
# Install package for extractive summarization
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.5 MB/s eta 0:00:00


In [ ]:
# Mount the drive if running on colab 
from google.colab import drive
drive.mount('/content/drive')
# Import other required libraries 
from transformers import pipeline
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
import math
import tqdm
tqdm.tqdm.pandas()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Function to generate the summary of a given string 
def get_from_model(transcript_string, model_name):
  result = model_name(transcript_string)[0]
  return result

In [1]:
# Function to split the transcript text into smaller chunks
# and aggregate the results of each chunk to generate the complete summary
def aggregate_summary_chunks(sentences,chunk_size):
  try:
    chunk_pos = 0
    num_sent = len(sentences)
    summarized_string = ""
    while chunk_pos <= num_sent:
      if chunk_pos+chunk_size <= num_sent:
        summarized_string += get_from_model(" ".join(sentences[chunk_pos:chunk_pos+chunk_size]), model_pipe)['summary_text'] + " "
        chunk_pos+=chunk_size
      else:
        summarized_string += get_from_model(" ".join(sentences[chunk_pos:]), model_pipe)['summary_text']
        chunk_pos=num_sent+1
    return summarized_string
  except Exception as e:
    res_str = "My error note: Failed at chunk size "+str(chunk_size)
    return res_str
  

In [ ]:
# Read the required data
read_data = pd.read_excel('/content/drive/MyDrive/summary_3_and_25_complete.xlsx')

In [ ]:
# Initialize the model
# device = 0 when we want to use GPU
model_pipe = pipeline("summarization", model="facebook/bart-large-xsum", device=0)

In [ ]:
# Uncomment the commented portion only after failure and to continue from the point of failure
# Replace the number 15 with the size at which failure happened
data_to_test = read_data#[read_data['fb_transcript_summary']=='My error note: Failed at chunk size 15']

In [ ]:
data_to_test.head(1)

,xml_filepath,title,summary,description,duration_minutes,duration_seconds,show_filename_prefix,episode_filename_prefix,json_filepath,json_transcript,summary_25,summary_3,summary_35,fb_transcript_summary,fb_summary_25,fb_summary_3
0,/content/drive/MyDrive/TREC/spotify-podcasts-2...,Ep.05 Why We Don't Sell Presets,<p>Today on the podcast we go on a JOURNEY! We...,<p>Today on the podcast we go on a JOURNEY! We...,3256,54.2712,show_015DbLwcXu2fK7e9jIfbFo,74t5WREXUbhEKNI89CNSkL,/content/drive/MyDrive/TREC/spotify-podcasts-2...,Welcome back to another episode of tuxedo tim...,Welcome back to another episode of tuxedo time...,Welcome back to another episode of tuxedo time...,Welcome back to another episode of tuxedo time...,We're back with another episode of Tuxedos. It...,It's time for another episode of Tuxedos. We'r...,"In our last episode of Tuxedos, we were sittin..."


In [ ]:
# Uncomment the commented portion only after failure and to continue from the point of failure
# fail = data_to_test.head(1)
# Replace the number 10 with the size at which you want to handle the failed record
# fail['fb_transcript_summary'] = fail['json_transcript'].progress_apply(lambda x: aggregate_summary_chunks(sent_tokenize(x), 10))
# success_data = read_data[read_data['fb_transcript_summary']!='My error note: Failed at chunk size 15'].append(fail, ignore_index=True)
# data_to_test = data_to_test.tail(data_to_test.shape[0]-1)

In [ ]:
# Code for sanity check to see if the generation is working
# summary_sentences = sent_tokenize(data_to_test['summary_25'].iloc[0])
# transcript_sentences = sent_tokenize(data_to_test['json_transcript'].iloc[0])
# abs_sum = aggregate_summary_chunks(summary_sentences,5)
# trans_sum = aggregate_summary_chunks(transcript_sentences,15)

In [ ]:
# Apply the function to the column from which we want to get the abstractive summary
# Replace 'fb_summary_35' with new column name to store the summaries
# Replace 'summary_35' with column name from which the summaries need to be identified
# Replace the number 5 with the chunk size you want the model to consider
data_to_test['fb_summary_35'] = data_to_test['summary_35'].progress_apply(lambda x: aggregate_summary_chunks(sent_tokenize(x), 5))

  0%|          | 4/917 [00:27<1:48:33,  7.13s/it]Your max_length is set to 62, but you input_length is only 59. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=29)
Your max_length is set to 62, but you input_length is only 3. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
  2%|▏         | 14/917 [01:44<1:45:08,  6.99s/it]Your max_length is set to 62, but you input_length is only 56. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)
Your max_length is set to 62, but you input_length is only 3. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
  3%|▎         | 29/917 [02:58<45:02,  3.04s/it]Your max_length is set to 62, but you input_length is only 57. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)
Your max_length is set to 62, but you input_length is only 3. You might consider decreasing max_l

In [ ]:
# view the data
data_to_test.head()

,xml_filepath,title,summary,description,duration_minutes,duration_seconds,show_filename_prefix,episode_filename_prefix,json_filepath,json_transcript,summary_25,summary_3,summary_35,fb_transcript_summary,fb_summary_25,fb_summary_3,fb_summary_35
0,/content/drive/MyDrive/TREC/spotify-podcasts-2...,Ep.05 Why We Don't Sell Presets,<p>Today on the podcast we go on a JOURNEY! We...,<p>Today on the podcast we go on a JOURNEY! We...,3256,54.271200,show_015DbLwcXu2fK7e9jIfbFo,74t5WREXUbhEKNI89CNSkL,/content/drive/MyDrive/TREC/spotify-podcasts-2...,Welcome back to another episode of tuxedo tim...,Welcome back to another episode of tuxedo time...,Welcome back to another episode of tuxedo time...,Welcome back to another episode of tuxedo time...,We're back with another episode of Tuxedos. It...,It's time for another episode of Tuxedos. We'r...,"In our last episode of Tuxedos, we were sittin...",It's been a while since we've recorded a podca...
1,/content/drive/MyDrive/TREC/spotify-podcasts-2...,How to Grow as an Esports Player - The Wavelen...,Ever wanted a podcast from your three favorite...,Ever wanted a podcast from your three favorite...,4650,77.500850,show_01DbRiALDPdvZdoiY8yQL6,5fG4VlWnWwzAt6mSs0H7lY,/content/drive/MyDrive/TREC/spotify-podcasts-2...,"What's up, guys? This episode of the podcast ...",This episode of the podcast is brought to you ...,This episode of the podcast is brought to you ...,This episode of the podcast is brought to you ...,It's time for this week's episode of the F m--...,On this episode of the podcast we're talking a...,On this episode of the podcast we're talking a...,This week we're talking about how to make a po...
2,/content/drive/MyDrive/TREC/spotify-podcasts-2...,Irish Illustrated Insider: Observations and an...,<p>The Irish Illustrated Insider crew discusse...,<p>The Irish Illustrated Insider crew discusse...,3415,56.930300,show_01Txd706SjsgvM0cm0UXuM,5hvOWPoB0j6HMrSVAMtJLV,/content/drive/MyDrive/TREC/spotify-podcasts-2...,You are listening to Irish illustrate Insider...,You are listening to Irish illustrate Insider....,You are listening to Irish illustrate Insider....,You are listening to Irish illustrate Insider....,Notre Dame's first spring practice took place ...,Notre Dame quarterback Brandon Wimbush threw t...,"It didn't feel like a big Revelation either, I...",Notre Dame head coach Pat Narduzzi has spoken ...
3,/content/drive/MyDrive/TREC/spotify-podcasts-2...,Irish Illustrated Insider: Previewing Notre Da...,<p>Irish Illustrated Insider tackles NFL Combi...,<p>Irish Illustrated Insider tackles NFL Combi...,2808,46.802917,show_01Txd706SjsgvM0cm0UXuM,7JG3lLnRoDdOxuqjf14ZkM,/content/drive/MyDrive/TREC/spotify-podcasts-2...,You have tuned into Irish Illustrated Insider...,You have tuned into Irish Illustrated Insider....,You have tuned into Irish Illustrated Insider....,You have tuned into Irish Illustrated Insider....,It's time for the latest edition of the Irish ...,If you have been listening to the Irish Illust...,If you have been listening to the Irish Illust...,"It's been a while since we've done a podcast, ..."
4,/content/drive/MyDrive/TREC/spotify-podcasts-2...,Episode 73 - Comeback SZN,<p>Breaking down a classic Calgary Flames come...,<p>Breaking down a classic Calgary Flames come...,2969,49.487233,show_01eumErJvBdxCW4YJivbwc,2WQ1GcC6J0k7qsO8Vvf2be,/content/drive/MyDrive/TREC/spotify-podcasts-2...,"What's up, everybody? Welcome to the in the d...",Welcome to the in the dome podcast podcast bod...,Welcome to the in the dome podcast podcast bod...,Welcome to the in the dome podcast podcast bod...,It's time for the latest episode of the In the...,A couple of weeks ago I was getting ready for ...,It's been a while since I've done a podcast an...,It's been a long time since I've been on a pod...


In [ ]:
# Uncomment the commented portion only after failure and to continue from the point of failure
# final_data = success_data.append(data_to_test, ignore_index=True)
# final_data[final_data['fb_transcript_summary']=='My error note: Failed at chunk size 15']
# to_be_stored = final_data.iloc[: , 1:]
# data_to_test = to_be_stored

In [ ]:
# Store data 
data_to_test.to_excel('/content/drive/MyDrive/summary_all_complete.xlsx', index=False)